In [1]:
import pandas as pd

In [2]:
biz_df = pd.read_csv('biz_cluster.csv')

In [6]:
biz_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 57173 entries, 0 to 57172
Data columns (total 24 columns):
Unnamed: 0                57173 non-null int64
business_id               57173 non-null object
RestaurantsPriceRange2    51111 non-null float64
stars                     57173 non-null float64
review_count              57173 non-null int64
categories                57173 non-null object
cl_0                      25785 non-null float64
cl_1                      37611 non-null float64
cl_2                      6306 non-null float64
cl_3                      327 non-null float64
cl_4                      11547 non-null float64
cl_5                      14129 non-null float64
cl_6                      1 non-null float64
cl_7                      3366 non-null float64
cl_8                      2667 non-null float64
cl_9                      2073 non-null float64
cl_10                     380 non-null float64
cl_11                     2937 non-null float64
cl_12                     47